# K-Means Algorithm

This notebook will present the development of the k-means algorithm done by my incredible self !

The goal here is not to use scikit-learn straight ahead, but rather to learn by myself how k-means works under the hood and what are the different mechanisms ruling it.

So Now lets go baby !

In [1]:
import numpy
import sklearn
from sklearn import datasets

### Initialization ###

# First set k
k = 3
# Retrieve the data
X = datasets.load_iris().data
# First initialization of mu (the matrix of centroids)
n_rows = X.shape[0]
random_indices = numpy.random.choice(n_rows,k,replace=False)
mu = X[random_indices,:] # (k,4) matrix
# Affectation matrix
A = numpy.zeros((X.shape[0],k))
# Distance matrix distances[i,j] is the distance between the datapoint i and the centroid j.
distances = numpy.zeros((len(X),k))

def calc_affectation_matrix(debug: bool = False) -> numpy.ndarray:
    # First, compute the distances of every datapoint to every centroides
    for i in range(len(X)):
        for j in range(k):
            distances[i,j] = numpy.sum(numpy.square(X[i] - mu[j]))
    # Print that just for debug purpose
    if debug: print(distances, distances.shape)
    
    # Then, retrieve the argmin (ie. the index of the smallest element in each row)
    argmin = numpy.argmin(distances, axis=1)
    # Print that just for debug purpose
    if debug: print(argmin, argmin.shape)
    
    # Initialize an affectation matrix to 0 of shape (n,k)
    A = numpy.zeros((X.shape[0],k))
    for i, j in enumerate(argmin):
        # Set only the smallest element to 1
        A[i,j] = 1
      
    # Return it  
    return A
    
def calc_centroïds_matrix():
    # Update every centroids j
    for j in range(k):
        sum_vector_cluster_j = 0
        div = 0
        for i in range(n_rows):
            sum_vector_cluster_j += A[i,j] * X[i]
            div += A[i,j]
        mu[j] = sum_vector_cluster_j / div
        
def loss_function():
    loss = 0
    for i in range(n_rows):
        for j in range(k):
            if A[i,j]:
                loss += distances[i,j]
    return loss

def verif():
    possible_mappings = [[0]*3 for _ in range(3)]
    a = numpy.argmax(A, axis=1)
    b = datasets.load_iris().target
    for el_a, el_b in zip(a, b):
        possible_mappings[el_a][el_b] += 1
        
    mapping = numpy.argmax(possible_mappings, axis=1)
    return sum(mapping[predicted] == b[i] for i, predicted in enumerate(a))
       
previous_loss = -1
curr_loss = loss_function()
while curr_loss != previous_loss:
    print(mu, loss_function())
    print()
    A = calc_affectation_matrix()
    calc_centroïds_matrix()
    previous_loss, curr_loss = curr_loss, loss_function()
    
print(f"{verif()} predictions were correct out of {n_rows} datapoints")


[[4.4 3.2 1.3 0.2]
 [6.7 3.3 5.7 2.1]
 [4.8 3.4 1.6 0.2]] 0

[[4.5        3.05555556 1.28888889 0.2       ]
 [6.33763441 2.90430108 5.01827957 1.72258065]
 [5.1375     3.35416667 1.77916667 0.37291667]] 246.3799999999999

[[4.66111111 3.08888889 1.36666667 0.2       ]
 [6.31458333 2.89583333 4.97395833 1.703125  ]
 [5.17777778 3.47222222 1.71111111 0.35555556]] 148.1679673408775

[[4.7        3.10952381 1.39047619 0.2       ]
 [6.30103093 2.88659794 4.95876289 1.69587629]
 [5.20625    3.540625   1.671875   0.35      ]] 145.78479914158947

[[4.7        3.10952381 1.39047619 0.2       ]
 [6.30103093 2.88659794 4.95876289 1.69587629]
 [5.20625    3.540625   1.671875   0.35      ]] 145.45269176485036

100 predictions were correct out of 150 datapoints


Distance euclidienne : 4.0
